# Graph Algorithms

## Graph, stack, queue & priority queue implementations

Using adjacency lists for graphs, singly-linked list for stacks, doubly-linked list for queues, and heap for priority queues. See `Graph (Adjacency Lists).ipynb`, `Linked List.ipynb` and `Heap.ipynb` for details

In [1]:
import collections

Edge = collections.namedtuple('Edge', ['from_', 'to', 'weight'])

class Graph:
    """Graph implemented with adjacency lists.
    
    For conciseness, edge/vertex removal.
    """
  
    def __init__(self, n=0, directed=True):
        """Constructs a graph with n nodes and 0 edges."""
        self._vertex_to_last_edge = [-1] * n # size n
        self._edge_to_destination = [] # size m
        self._edge_to_next_edge = [] # size m
        self._weights = [] # size m
        self.n = n
        self.m = 0
        self.directed = directed
        
    def display(self):
        """Prints the adjacency lists in a human readable way."""
        print()
        print('{:^8s} | {:^8s}'.format('Vertex', 'Neighbors'))
        print('---------------------------------------------')
        for v in range(self.n):
            print('{:^8d} | {}'.format(v, ', '.join(map(str, sorted(self.neighbors(v))))))
        print()
      
    def neighbors(self, i):
        """Returns the nodes directly accessible from node i."""
        self._validate(i)
        edge = self._vertex_to_last_edge[i]
        adjacent_vertices = []
        while True:
            if edge == -1:
                return sorted(adjacent_vertices)
            adjacent_vertices.append(self._edge_to_destination[edge])
            edge = self._edge_to_next_edge[edge]
            
    def add_edge(self, i, j, weight=1.0, other_side_too=None):
        """Adds an edge from node i to node j."""
        self._validate(i, j)
        if j not in self.neighbors(i):
            self._edge_to_destination.append(j)
            last_edge = self._vertex_to_last_edge[i] # can be -1
            self._edge_to_next_edge.append(last_edge)
            self._vertex_to_last_edge[i] = self.m
            self._weights.append(weight)
            self.m += 1
            if not self.directed and other_side_too is None and i != j:
                self.add_edge(j, i, weight, other_side_too=False)

    def add_vertex(self):
        """Adds a vertex."""
        self._vertex_to_last_edge.append(-1)
        self.n += 1
        
    def edges(self, v=None):
        """Returns list of edges of the graph, or of vertex v if not None."""
        if v is not None:
            self._validate(v)
            edges = []
            edge = self._vertex_to_last_edge[v]
            while edge != -1:
                to = self._edge_to_destination[edge]
                edges.append(Edge(from_=v, to=to, weight=self._weights[edge]))
                edge = self._edge_to_next_edge[edge]
            return edges
        else:
            edges = []
            for v in range(self.n):
                edge = self._vertex_to_last_edge[v]
                while edge != -1:
                    to = self._edge_to_destination[edge]
                    if self.directed or v <= to:
                        edges.append(Edge(from_=v, to=to, weight=self._weights[edge]))
                    edge = self._edge_to_next_edge[edge]
            return edges

    def _validate(self, *args):
        """Validates index."""
        for i in args:
            if i >= self.n or i < 0:
                raise ValueError('Edges must be between 0 and %d' % (self.n - 1))

In [2]:
class StackNode:
    """Node of a linear singly-linked list."""
    
    def __init__(self, value):
        self.value = value
        self.next = None


class Stack:
    """A stack."""
    
    def __init__(self):
        self.next = None
        
    def __iter__(self):
        node = self
        while node.next:
            node = node.next
            yield node.value
    
    def push(self, value):
        node = StackNode(value)
        node.next = self.next
        self.next = node
    
    def pop(self):
        if not self.next:
            raise ValueError('Empty stack')
        node = self.next
        self.next = self.next.next
        return node.value
    
    def peek(self):
        if not self.next:
            raise ValueError('Empty stack')
        return self.next.value
    
    def empty(self):
        return not self.next

In [3]:
class QueueNode:
    """Node of a linear doubly-linked list."""
    
    def __init__(self, value):
        self.value = value
        self.next = None
        self.prev = None

class Queue:
    """Queue implemented as a doubly-linked list."""
    
    def __init__(self):
        self.front = None
        self.back = None
    
    def enqueue(self, value):
        """Add element to the back."""
        node = QueueNode(value)
        if not self.back:
            # Empty queue.
            self.back = node
            self.front = node
        else:
            self.back.next = node
            node.prev = self.back
            self.back = node
    
    def dequeue(self):
        """Remove element from the front."""
        if not self.back:
            raise ValueError('Empty queue')
        node = self.front
        if not self.front.next:
            # Queue is now empty.
            self.front = None
            self.back = None
        else:
            self.front.next.prev = None
            self.front = self.front.next
        return node.value
    
    def empty(self):
        return not self.back

In [4]:
class PriorityQueue:
    """A min-heap."""
    
    def __init__(self):
        self.array = []
        self.len = 0
        
    def items(self):
        for item in self.array:
            yield item[0]
            
    def empty(self):
        return not self.array
        
    def insert(self, value, priority):
        """Inserts an element."""
        self.array.append((value, priority))
        self.len += 1
        idx = self.len - 1
        while True:
            p = self._parent(idx)
            if p is None:
                return
            if self.array[p][1] <= priority:
                return
            if self.array[p][1] > priority:
                self.array[idx], self.array[p] = self.array[p], self.array[idx]
                idx = p

    def pop(self):
        """Removes and returns min element."""
        if self.len == 0:
            raise ValueError('Empty priority queue')
        if self.len == 1:
            self.len = 0
            return self.array.pop()[0]
        top = self.array[0]
        last = self.array.pop()
        self.array[0] = last
        self.len -= 1
        idx = 0
        while True:
            left = self._left(idx)
            right = self._right(idx)
            # No child, done.
            if left is None and right is None:
                break
            # One child, necessary on the left.
            elif right is None:
                val = self.array[left][1]
                if val < last[1]:
                    self.array[idx], self.array[left] = self.array[left], self.array[idx]
                    idx = left
                    continue
                else:
                    break
            # Two children. If necessary, sift down with the larger one.
            else:
                a = self.array[left][1]
                b = self.array[right][1]
                if a < b:
                    if a < last[1]:
                        self.array[idx], self.array[left] = self.array[left], self.array[idx]
                        idx = left
                        continue
                    else:
                        break
                else:
                    if b < last[1]:
                        self.array[idx], self.array[right] = self.array[right], self.array[idx]
                        idx = right
                        continue
                    else:
                        break
        return top[0]

    def _parent(self, i):
        if i == 0:
            return None
        return (i - 1) // 2

    def _left(self, i):
        idx = 2 * i + 1
        if idx >= self.len:
            return None
        return idx

    def _right(self, i):
        idx = 2 * i + 2
        if idx >= self.len:
            return None
        return idx

## Graph Traversal

### Depth-first search (DFS)

Traverse the graph by going down a branch until a "leaf" is reached and then backtracking. Implemented using a stack.

In [5]:
def depth_first_search(g, func, start_vertex=0):
    """Pre-order depth-first search."""
    s = Stack()
    s.push(start_vertex)
    seen = {start_vertex}
    while not s.empty():
        v = s.pop()
        func(v) # pre-order
        for n in g.neighbors(v):
            if n not in seen:
                s.push(n)
                seen.add(n)

In [6]:
g = Graph(10)
g.add_edge(0, 2)
g.add_edge(2, 3)
g.add_edge(2, 7)
g.add_edge(2, 8)
g.add_edge(4, 4)
g.add_edge(4, 5)
g.add_edge(5, 6)
g.add_edge(7, 2)
g.add_edge(7, 3)
g.add_edge(7, 4)
g.add_edge(7, 5)
g.add_edge(7, 6)
g.add_edge(8, 7)
g.add_edge(8, 9)
g.add_edge(9, 1)
g.display()

depth_first_search(g, lambda v: print('Visiting %d' % v))


 Vertex  | Neighbors
---------------------------------------------
   0     | 2
   1     | 
   2     | 3, 7, 8
   3     | 
   4     | 4, 5
   5     | 6
   6     | 
   7     | 2, 3, 4, 5, 6
   8     | 7, 9
   9     | 1

Visiting 0
Visiting 2
Visiting 8
Visiting 9
Visiting 1
Visiting 7
Visiting 6
Visiting 5
Visiting 4
Visiting 3


![Graph](img/graph_bfs_dfs.png)

### Breadth-first search (BFS)

Traverse the graph "level by level." Implemented using a queue.

In [7]:
def breadth_first_search(g, func, start_vertex=0):
    """Pre-order breadth-first search."""
    q = Queue()
    q.enqueue(start_vertex)
    seen = {start_vertex}
    while not q.empty():
        v = q.dequeue()
        func(v) # pre-order
        for n in g.neighbors(v):
            if n not in seen:
                q.enqueue(n)
                seen.add(n)

In [8]:
g.display()

breadth_first_search(g, lambda v: print('Visiting %d' % v))


 Vertex  | Neighbors
---------------------------------------------
   0     | 2
   1     | 
   2     | 3, 7, 8
   3     | 
   4     | 4, 5
   5     | 6
   6     | 
   7     | 2, 3, 4, 5, 6
   8     | 7, 9
   9     | 1

Visiting 0
Visiting 2
Visiting 3
Visiting 7
Visiting 8
Visiting 4
Visiting 5
Visiting 6
Visiting 9
Visiting 1


## Topological Sort (for DAGs)

Directed Acyclic Graphs are equivalent to graphs that have a topological sort, i.e. partial order. The goal is to list all vertices so that if there's an edge from u to v, u comes before v in the list.

The naïve way: take any node with no incoming edge. Remove outgoing edges. Repeat. It works but depending on the complexity of removing edges, it can be slow. Instead we can use a non-destructive algorithms that relies on queues.

Precompute deg(v) for each node v, which is the number of incoming edge to v. Put all the ones with dev(v) = 0 in a queue. Then take the front of the queue as the next element, decrease deg for all its neighbors, and if one reaches 0 then add it to the queue. Repeat until the queue is empty.

In [9]:
def topological_sort(g):
    """Topological sort for a DAG."""
    deg = [0] * g.n # contains number of incoming edges
    for v in range(g.n):
        for n in g.neighbors(v):
            deg[n] += 1
    
    sorted_vertices = []
    
    q = Queue() # contains vertices with deg(v) = 0
    for v in range(g.n):
        if deg[v] == 0:
            q.enqueue(v)
    
    while not q.empty():
        v = q.dequeue()
        sorted_vertices.append(v)
        for n in g.neighbors(v):
            deg[n] -= 1
            if deg[n] == 0:
                q.enqueue(n)
    
    return sorted_vertices

In [10]:
g = Graph(9)
g.add_edge(6, 2)
g.add_edge(2, 5)
g.add_edge(2, 8)
g.add_edge(1, 2)
g.add_edge(2, 7)
g.add_edge(1, 4)
g.add_edge(4, 8)
g.add_edge(3, 4)
g.add_edge(3, 7)

g.display()


 Vertex  | Neighbors
---------------------------------------------
   0     | 
   1     | 2, 4
   2     | 5, 7, 8
   3     | 4, 7
   4     | 8
   5     | 
   6     | 2
   7     | 
   8     | 



![Graph](img/graph_topological_sort.png)

In [11]:
print(topological_sort(g))

[0, 1, 3, 6, 4, 2, 5, 7, 8]


The order is better seen on this graph:

![Graph](img/graph_topological_sort_2.png)

## Minimum Spanning Tree (MST)

Goal: for G = (V, E) undirected, construct a tree using a subset of E that connects all vertices.

### Kruskal's algorithm

Idea 1: Edge with minimum weight must be included.

Idea 2: Once an edge is included, we merge the connected parts into supernodes, using Union-Find.

Union-Find uses a array of size |v|, initialized to A(v) = v, and supports two operations:
  * **Find:** two vertices are connected if A(u) = A(v)
  * **Union:** merge two groups by settings every A(u') where A(u') = A(u) to A(v)

In [12]:
def kruskal(g):
    edges = sorted(g.edges(), key=lambda e: e.weight)
    selected_edges = []
    a = list(range(g.n)) # A(v) = v
    for e in edges:
        if a[e.from_] != a[e.to]: # "find"
            # Not yet connected, let's do it now.
            selected_edges.append(e)
            old, new = a[e.to], a[e.from_]
            for i in range(g.n):
                if a[i] == old:
                    a[i] = new # "union"
        # Else, ignore the edge.
    k = Graph(g.n, directed=False)
    cost = 0.0
    for e in selected_edges:
        k.add_edge(e.from_, e.to, e.weight)
        cost += e.weight
    return k, cost

In [13]:
g = Graph(9, directed=False)

# Use weight = from + to for simplicity.
g.add_edge(0, 6, 6.0)
g.add_edge(1, 6, 7.0)
g.add_edge(1, 7, 8.0)
g.add_edge(2, 5, 7.0)
g.add_edge(2, 7, 9.0)
g.add_edge(2, 8, 10.0)
g.add_edge(3, 4, 7.0)
g.add_edge(3, 6, 9.0)
g.add_edge(3, 7, 10.0)
g.add_edge(4, 5, 9.0)
g.add_edge(5, 8, 13.0)
g.add_edge(7, 8, 15.0)
g.display()


 Vertex  | Neighbors
---------------------------------------------
   0     | 6
   1     | 6, 7
   2     | 5, 7, 8
   3     | 4, 6, 7
   4     | 3, 5
   5     | 2, 4, 8
   6     | 0, 1, 3
   7     | 1, 2, 3, 8
   8     | 2, 5, 7



![Graph](img/graph_mst.png)

In [14]:
k, cost = kruskal(g)
k.display()
print('Cost: %r' % cost)


 Vertex  | Neighbors
---------------------------------------------
   0     | 6
   1     | 6, 7
   2     | 5, 7, 8
   3     | 4, 6
   4     | 3
   5     | 2
   6     | 0, 1, 3
   7     | 1, 2
   8     | 2

Cost: 63.0


![Graph](img/graph_mst_solved.png)

### Prim's algorithm

* Pick any node and put it in S.
* Find edge e with smallest weight that connects a vertex in S to a vertex v not yet in S.
* Add e to MST and v to S.
* Repeat n times (until S = V).

In [15]:
def prims(g):
    u = 0 # start with node 0
    S = {u}
    edges = PriorityQueue() # edges connecting S and its complement
    selected_edges = []
    while len(S) != g.n:
        for e in g.edges(u):
            if e.to not in S:
                edges.insert(e, e.weight)
        while True:
            e = edges.pop()
            if e.to not in S:
                selected_edges.append(e)
                S.add(e.to)
                u = e.to
                break
    p = Graph(g.n, directed=False)
    cost = 0.0
    for e in selected_edges:
        p.add_edge(e.from_, e.to, e.weight)
        cost += e.weight
    return p, cost

In [16]:
p, cost = prims(g)
p.display()
print('Cost: %r' % cost)


 Vertex  | Neighbors
---------------------------------------------
   0     | 6
   1     | 6, 7
   2     | 5, 7, 8
   3     | 4, 6
   4     | 3
   5     | 2
   6     | 0, 1, 3
   7     | 1, 2
   8     | 2

Cost: 63.0


## Cycle detection

### Cycles in directed graph

For directed graphs, we can use DFS and make sure no node is present in the recursion stack that led to it.

In [17]:
def has_cycle(g, start_vertex=0):
    """Pre-order depth-first search."""
    s = Stack()
    s.push(start_vertex)
    seen = {start_vertex}
    visited = set()
    while not s.empty():
        # We need to keep the node in the chain so we'll only peek.
        v = s.peek()
        if v in visited:
            s.pop()
            continue
        visited.add(v)
        for n in g.neighbors(v):
            if n in s:
                # n is present in the recursion stack => cycle.
                return True
            if n not in seen:
                s.push(n)
                seen.add(n)
    return False

In [18]:
g = Graph(5)
g.add_edge(0, 1)
g.add_edge(1, 2)
g.add_edge(1, 4)
g.add_edge(2, 3)
g.add_edge(2, 4)
print(has_cycle(g))

False


![Graph](img/graph_directed_no_cycle.png)

In [19]:
g = Graph(5)
g.add_edge(0, 1)
g.add_edge(1, 2)
g.add_edge(2, 3)
g.add_edge(2, 4)
g.add_edge(4, 1)
print(has_cycle(g))

True


![Graph](img/graph_directed_cycle.png)

In [20]:
# One example with self link.
g = Graph(5)
g.add_edge(0, 1)
g.add_edge(1, 2)
g.add_edge(1, 4)
g.add_edge(2, 3)
g.add_edge(2, 4)
g.add_edge(3, 3)
print(has_cycle(g))

True


### Cycles in undirected graph

Here we could make sure that no neighbor (except for the node we're coming from) is in the `visited` set. But since it's also means `G` is a tree, assuming it is connected with `n` vertices, we can just check it has `n - 1` edges.

In [21]:
def has_cycle_undirected(g, start_vertex=0):
    return len(g.edges()) != g.n - 1

In [22]:
g = Graph(5, directed=False)
g.add_edge(0, 1)
g.add_edge(1, 2)
g.add_edge(2, 3)
g.add_edge(2, 4)
g.add_edge(4, 1)
print(has_cycle_undirected(g))

True


![Graph](img/graph_undirected_cycle.png)

In [23]:
g = Graph(5, directed=False)
g.add_edge(0, 1)
g.add_edge(1, 2)
g.add_edge(2, 3)
g.add_edge(4, 1)
print(has_cycle_undirected(g))

False


![Graph](img/graph_undirected_no_cycle.png)

## Shortest path

Problem: compute the shortest path from a node $S$ to all other nodes.

In [24]:
def print_shortest_paths(distance, predecessor):
    """Prints results."""
    for i, d in enumerate(distance):
        chain = [predecessor[i]]
        while chain[0] is not None:
            chain = [predecessor[chain[0]]] + chain
        print('%d: distance = %.1f via %s' % (i, d, ' -> '.join(map(str, chain[1:] + [i]))))

### Bellman-Ford algorithm (general solution for directed graphs with negative weights)

This works even with negative weights, and detects negative cycles. Runs in O(VE).

Idea: Start with $dist(S, u) = \inf$ for all nodes. Then repeat $\left|V\right|$ times: for each edge $(u, v)$ of weight $w$, update $dist(S, v) = min(dist(S, u) + w, dist(S, v))$.

We need to repeat it $\left|V\right|$ times because in the worst case there are $\left|V\right|$ updates needed (or really $\left|V\right| - 1$).

If there's a negative cycle, then at the end we'll find an edge $(u, v) = w$ such that $dist(v) < dist(u) + w$.

In [25]:
def bellman_ford(g, source=0):
    # Initialization.
    distance = [float('inf')] * g.n
    distance[source] = 0.0
    predecessor = [None] * g.n
    
    # Repeat |V| - 1 times.
    for _ in range(g.n - 1):
        for e in g.edges():
            if distance[e.from_] + e.weight < distance[e.to]:
                distance[e.to] = distance[e.from_] + e.weight
                predecessor[e.to] = e.from_
    
    # Check for negative cycles.
    for e in g.edges():
        if distance[e.from_] + e.weight < distance[e.to]:
            chain = [e.to]
            while chain[0] not in chain[1:]:
                chain = [predecessor[chain[0]]] + chain
            raise ValueError('Negative cycle detected: %s...' % ' -> '.join(map(str, chain)))
    
    return distance, predecessor

In [26]:
g = Graph(4)
g.add_edge(0, 1, 2.0)
g.add_edge(0, 2, -1.0)
g.add_edge(1, 2, 3.0)
g.add_edge(2, 1, -2.0)

distance, predecessor = bellman_ford(g)
print_shortest_paths(distance, predecessor)

0: distance = 0.0 via 0
1: distance = -3.0 via 0 -> 2 -> 1
2: distance = -1.0 via 0 -> 2
3: distance = inf via 3


In [27]:
g = Graph(4)
g.add_edge(0, 1, 2.0)
g.add_edge(0, 2, -1.0)
g.add_edge(1, 2, 3.0)
g.add_edge(2, 1, -4.0) # negative cycle 1->2->1->...

try:
    distance, predecessor = bellman_ford(g)
except ValueError as e:
    print(str(e))

Negative cycle detected: 2 -> 1 -> 2...


In [28]:
# More complex example.
g = Graph(6)
g.add_edge(0, 1, 1.0)
g.add_edge(0, 1, -4.0) # note, two edges between 0 and 1 with different weights
g.add_edge(0, 5, -1.0)
g.add_edge(1, 2, 2.0)
g.add_edge(1, 3, 4.0)
g.add_edge(1, 4, 10.0)
g.add_edge(2, 3, 3.0)
g.add_edge(3, 1, -4.0)
g.add_edge(3, 4, 6.0)
g.add_edge(4, 3, -3.0)
g.add_edge(5, 4, -2.0)

print_shortest_paths(*bellman_ford(g))

0: distance = 0.0 via 0
1: distance = -10.0 via 0 -> 5 -> 4 -> 3 -> 1
2: distance = -8.0 via 0 -> 5 -> 4 -> 3 -> 1 -> 2
3: distance = -6.0 via 0 -> 5 -> 4 -> 3
4: distance = -3.0 via 0 -> 5 -> 4
5: distance = -1.0 via 0 -> 5


### Dijkstra's algorithm: Undirected and directed with no negative cycles

It uses a min-priority queue to perform a best-first search. If we're only interested in one destination we don't need to traverse the entire graph. It runs in O(E + V log V) since removing the min from a heap is O(log V), and we do it for each vertex; and every edge is considered once.

In [29]:
def dijkstra(g, start_vertex=0, end_vertex=None):
    # Initialization.
    distance = [float('inf')] * g.n
    predecessor = [None] * g.n
    distance[start_vertex] = 0
    visited = set() # fully visited with their final distance
    q = PriorityQueue()
    q.insert(start_vertex, 0)
    
    while not q.empty():
        u = q.pop() # O(log V)
        if u in visited:
            continue  # need to do this because q.insert() doesn't replace
        if u == end_vertex:
            return distance[u], predecessor # we can stop here
        visited.add(u)

        for e in g.edges(u):
            v = e.to
            if v not in visited:
                if distance[u] + e.weight < distance[v]:
                    distance[v] = distance[u] + e.weight
                    predecessor[v] = u
                    q.insert(v, distance[v])

    return distance, predecessor

In [30]:
# Simple.
g = Graph(4)
g.add_edge(0, 1, 2.0)
g.add_edge(0, 2, 1.0)
g.add_edge(1, 2, 3.0)
g.add_edge(2, 1, 4.0)

distance, predecessor = dijkstra(g)
print_shortest_paths(distance, predecessor)

0: distance = 0.0 via 0
1: distance = 2.0 via 0 -> 1
2: distance = 1.0 via 0 -> 2
3: distance = inf via 3


In [31]:
# Simple, with negative cycle: doesn't work!
g = Graph(4)
g.add_edge(0, 1, 2.0)
g.add_edge(0, 2, -1.0)
g.add_edge(1, 2, 3.0)
g.add_edge(2, 1, -4.0) # negative cycle 1->2->1->...

distance, predecessor = dijkstra(g)
print_shortest_paths(distance, predecessor)

0: distance = 0.0 via 0
1: distance = -5.0 via 0 -> 2 -> 1
2: distance = -1.0 via 0 -> 2
3: distance = inf via 3


In [32]:
# More complex example, with no negative cycle.
g = Graph(6)
g.add_edge(0, 1, 1.0)
g.add_edge(0, 1, -4.0) # note, two edges between 0 and 1 with different weights
g.add_edge(0, 5, -1.0)
g.add_edge(1, 2, 2.0)
g.add_edge(1, 3, 4.0)
g.add_edge(1, 4, 10.0)
g.add_edge(2, 3, 3.0)
g.add_edge(3, 1, -4.0)
g.add_edge(3, 4, 6.0)
g.add_edge(4, 3, -3.0)
g.add_edge(5, 4, -2.0)

print_shortest_paths(*dijkstra(g))

0: distance = 0.0 via 0
1: distance = -10.0 via 0 -> 5 -> 4 -> 3 -> 1
2: distance = -8.0 via 0 -> 5 -> 4 -> 3 -> 1 -> 2
3: distance = -6.0 via 0 -> 5 -> 4 -> 3
4: distance = -3.0 via 0 -> 5 -> 4
5: distance = -1.0 via 0 -> 5


### Application: snake and ladder

From https://www.geeksforgeeks.org/snake-ladder-problem-2/

Player can throw a 1-6 die, and has to go up when finding a ladder and down when finding a snake. What's the fastest path to 30?

![Board](img/snake_and_ladder.jpg)

In [42]:
board = {
    3: 22,
    5: 8,
    11: 26,
    17: 4,
    19: 7,
    20: 29,
    21: 9,
    27: 1,
}
g = Graph(31)
for i in range(1, 31):
    for d in range(1, 7):
        if i + d <= 30:
            g.add_edge(i, i + d)
            if i + d in board:
                g.add_edge(i + d, board[i + d], 0.0)
g.display()
dist, pred = dijkstra(g, 1)
chain = [30]
while chain[0] is not None:
    chain = [pred[chain[0]]] + chain
print('Fastest: %d dice throws via %s' % (dist[30], ' -> '.join(map(str, chain[1:]))))


 Vertex  | Neighbors
---------------------------------------------
   0     | 
   1     | 2, 3, 4, 5, 6, 7
   2     | 3, 4, 5, 6, 7, 8
   3     | 4, 5, 6, 7, 8, 9, 22
   4     | 5, 6, 7, 8, 9, 10
   5     | 6, 7, 8, 9, 10, 11
   6     | 7, 8, 9, 10, 11, 12
   7     | 8, 9, 10, 11, 12, 13
   8     | 9, 10, 11, 12, 13, 14
   9     | 10, 11, 12, 13, 14, 15
   10    | 11, 12, 13, 14, 15, 16
   11    | 12, 13, 14, 15, 16, 17, 26
   12    | 13, 14, 15, 16, 17, 18
   13    | 14, 15, 16, 17, 18, 19
   14    | 15, 16, 17, 18, 19, 20
   15    | 16, 17, 18, 19, 20, 21
   16    | 17, 18, 19, 20, 21, 22
   17    | 4, 18, 19, 20, 21, 22, 23
   18    | 19, 20, 21, 22, 23, 24
   19    | 7, 20, 21, 22, 23, 24, 25
   20    | 21, 22, 23, 24, 25, 26, 29
   21    | 9, 22, 23, 24, 25, 26, 27
   22    | 23, 24, 25, 26, 27, 28
   23    | 24, 25, 26, 27, 28, 29
   24    | 25, 26, 27, 28, 29, 30
   25    | 26, 27, 28, 29, 30
   26    | 27, 28, 29, 30
   27    | 1, 28, 29, 30
   28    | 29, 30
   29    | 30
   

### For a DAG

Step 1: topological sort in O(V+E)

Step 2: visit nodes in topological order and for every edge (u,v)=w do dist(v) = min(dist(v), dist(u)+w) which is also O(V+E)

In [34]:
def shortest_path_dag(g, start_vertex=0):
    # Step 1. Topological sort.
    a = topological_sort(g)
    
    # Step 2. Visit vertices in topological order.
    distance = [float('inf')] * g.n
    predecessor = [None] * g.n
    distance[start_vertex] = 0
    for u in a:
        for e in g.edges(u):
            v, w = e.to, e.weight
            if distance[u] + w < distance[v]:
                distance[v] = distance[u] + w
                predecessor[v] = u
    
    return distance, predecessor

In [43]:
g = Graph(9)
g.add_edge(6, 2)
g.add_edge(2, 5)
g.add_edge(2, 8)
g.add_edge(1, 2, 10.0)
g.add_edge(2, 7)
g.add_edge(1, 4)
g.add_edge(4, 8)
g.add_edge(3, 4)
g.add_edge(3, 7)

![Graph](img/graph_topological_sort_2.png)

In [44]:
print_shortest_paths(*shortest_path_dag(g, 1))

0: distance = inf via 0
1: distance = 0.0 via 1
2: distance = 10.0 via 1 -> 2
3: distance = inf via 3
4: distance = 1.0 via 1 -> 4
5: distance = 11.0 via 1 -> 2 -> 5
6: distance = inf via 6
7: distance = 11.0 via 1 -> 2 -> 7
8: distance = 2.0 via 1 -> 4 -> 8


## Max flow problem

A flow network is a directed graph where weights $c(u,v)$ represent capacity. Flows $f(u,v)$ must be lower than or equal to the capacity, and the sum of flows entering a node must be equal to the sum of flows leaving it, except for the source $s$ and sink $t$ for which $\sum_u f(s,u) = \sum_u f(u, t)$. The "flow" is the sum of all flows, i.e. $\left|f\right| = \sum_{(u,v) \in E} f(u,v)$, and we want to maximize it.

It's solved by the Ford-Fulkerson method, or more precisely its implementation the Edmonds-Karp algorithm. As long as there is a path from $s$ to $t$ in the residual graph (graph with capacity $c' = c - f$ and no flow), called an augmenting path, we add the minimum capacity on that path to the flow.

In [37]:
def ford_fulkerson_edmonds_karp(g, s, t):
    """Edmonds-Karp implementation of the Ford-Fulkerson method.
    
    Args:
        g: Directed graph where edges' weight represent the capacity.
        s: Source node.
        t: Sink node.
    
    Returns:
        The flow, i.e. f[u][v] for every edge (u, v).
    """
    f = [[0] * g.n for _ in range(g.n)]
    
    while True:
        # Is there a path from s to t in the residual graph? Let's find out with breadth-first search.
        predecessor = [(None, float('inf'))] * g.n
        q = Queue()
        q.enqueue(s)
        seen = {s}
        while not q.empty():
            u = q.dequeue()
            for e in g.edges(u):
                v, c = e.to, e.weight
                c_ = c - f[u][v] # residual capacity
                if c_ > 0 and v not in seen:
                    q.enqueue(v)
                    seen.add(v)
                    predecessor[v] = u, c_
                    if v == t:
                        # Found a path.
                        path = []
                        node = (t, None)
                        while node[0] != s:
                            pred = predecessor[node[0]]
                            path = [(pred[0], node[0], pred[1])] + path
                            node = pred
                        break
            else: # no break => no path found yet
                continue
            # Path found.
            min_capacity = min(path, key=lambda x: x[2])[2]
            print('Found path = %s with capacity %d' % ('-'.join([str(x[0]) for x in path] + [str(t)]), min_capacity))
            for e in path:
                f[e[0]][e[1]] += min_capacity
                f[e[1]][e[0]] -= min_capacity
            break
        else: # no break => no path found after exhausting BFS
            break
    
    return f

In [38]:
g = Graph(9)
g.add_edge(0, 1, 3)
g.add_edge(0, 2, 5)
g.add_edge(1, 3, 5)
g.add_edge(1, 4, 2)
g.add_edge(2, 4, 4)
g.add_edge(3, 6, 2)
g.add_edge(4, 5, 2)
g.add_edge(4, 6, 3)
g.add_edge(5, 7, 1)
g.add_edge(6, 8, 5)
g.add_edge(7, 8, 1)

In [39]:
flow = ford_fulkerson_edmonds_karp(g, 0, 8)

Found path = 0-2-4-6-8 with capacity 3
Found path = 0-1-3-6-8 with capacity 2
Found path = 0-2-4-5-7-8 with capacity 1


In [40]:
def print_flow(g, flow):
    total = 0
    for e in g.edges():
        f = flow[e.from_][e.to]
        print('%d -> %d: %d / %d' % (e.from_, e.to, f, e.weight))
        total += f
    print('Flow = %d' % total)

In [41]:
print_flow(g, flow)

0 -> 2: 4 / 5
0 -> 1: 2 / 3
1 -> 4: 0 / 2
1 -> 3: 2 / 5
2 -> 4: 4 / 4
3 -> 6: 2 / 2
4 -> 6: 3 / 3
4 -> 5: 1 / 2
5 -> 7: 1 / 1
6 -> 8: 5 / 5
7 -> 8: 1 / 1
Flow = 25


![Max-Flow](img/graph_maxflow.png)